In [ ]:
import numpy as np

In [ ]:
def encrypt(text,key):
  plain_text = text.upper().replace(" ","")
  key_size = key.shape[0]

  padding = (key_size - len(plain_text)) % key_size

  plain_text += 'X' * padding
  plain_arr = []
  for i in range(0,len(plain_text),key_size):
    plain_arr += [plain_text[i:i+key_size]]

  cipher_text = ""
  for ele in plain_arr:
    plain_num_arr = np.array([ord(c) - 65 for c in ele])

    encrypt_vec = np.matmul(plain_num_arr,key) % 26

    cipher_text += "".join(chr(value + 65) for value in encrypt_vec)

  return cipher_text


In [ ]:
def hill_inverse(key_matrix):
    def inverse_matrix(determinant, adjoint_matrix):
        inverse = (determinant * adjoint_matrix) % 26
        return inverse

    def cofactor_matrix(matrix):
        nrows, ncols = matrix.shape
        cofactors = np.zeros_like(matrix)
        for i in range(nrows):
            for j in range(ncols):
                sub_matrix = np.delete(np.delete(matrix, i, axis=0), j, axis=1)
                cofactors[i, j] = (-1) ** (i + j) * calculate_determinant(sub_matrix)
        return cofactors

    def calculate_determinant(matrix):
        n = matrix.shape[0]
        if n == 1:
            return matrix[0, 0]
        else:
            det = 0
            for j in range(n):
                sub_matrix = np.delete(matrix, 0, axis=0)
                sub_matrix = np.delete(sub_matrix, j, axis=1)
                det += (-1) ** j * matrix[0, j] * calculate_determinant(sub_matrix)
            return det % 26

    def inverse_determinant(determinant):
        for i in range(1, 26):
            mod_value = (determinant * i) % 26
            if mod_value == 1:
                break
        return i

    determinant = (np.linalg.det(key_matrix)) % 26
    adjoint_matrix = np.transpose(cofactor_matrix(key_matrix)) % 26
    inv_determinant = inverse_determinant(determinant)
    inverse = inverse_matrix(inv_determinant, adjoint_matrix)
    return inverse

def decrypt(ciphertext, key_matrix):
    key_matrix_inverse = hill_inverse(key_matrix)
    key_matrix_inverse = (key_matrix_inverse.round().astype(int) % 26)
    blocks = [ciphertext[i:i+key_matrix_inverse.shape[0]] for i in range(0, len(ciphertext), key_matrix_inverse.shape[0])]
    plaintext=""
    for block in blocks:
        block_vector = np.array([ord(c) - 65 for c in block])
        decrypted_vector = np.matmul(block_vector, key_matrix_inverse) % 26
        plaintext += "".join(chr(value + 65) for value in decrypted_vector)
    while plaintext.endswith('X'):
        plaintext = plaintext[:-1]
    return plaintext



In [ ]:
key = np.array([[17,17,5], [21,18,21],[2,2,19]])
plain_text = "pay more money"
encrypted_text = encrypt(plain_text, key)
decrypted_text = decrypt(encrypted_text, key)

In [ ]:
print("Plain Text:", plain_text)
print("Encrypted Text:", encrypted_text)
print("Decrypted Text:", decrypted_text)

Plain Text: pay more money
Encrypted Text: RRLMWBKASPDH
Decrypted Text: PAYMOREMONEY
